## Jobs scraping

This notebook extracts job offers from elempleo.com

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import wordcloud
from selenium import webdriver

page = 'http://www.elempleo.com/co/ofertas-empleo/'

page = requests.get(page)
soup = BeautifulSoup(page.content, 'html.parser')

The page presents all the articles in a container of class "main-content". Within that container, news articles are in boxes of the class "mh-posts-list-content".

We extract the code in the main container and then find all the articles. We display the code for the first article.

In [54]:
# Offers are in the third container row instance
table = soup.select(".container .row")[2]
#print(table)

jobs = soup.select(".result-item")
print(jobs[0])

<div class="result-item">
<div class="js-area-bind area-bind" data-url="/co/ofertas-trabajo/ejecutivo-comercial-externo-grupo-aval-/1883346871">
<img alt="Ofertas de empleo en Ventas y Servicios S.A.." class="hidden-xs img-responsive result-image" height="60" src="http://www.elempleo.com/colombia/Files/Companies/Logos/1317.gif" title="Ventas y Servicios S.A." width="130"/>
<ul class="list-unstyled result-info">
<li>
<h2 class="h4 item-title ee-mod">
<a class="text-ellipsis" href="/co/ofertas-trabajo/ejecutivo-comercial-externo-grupo-aval-/1883346871" title="Ejecutivo comercial externo grupo aval $951.000!!">
              Ejecutivo comercial externo grupo aval $951.000!!
            </a>
</h2>
<span class="hide">
            COP
          </span>
</li>
<li>
<h3 class="company-name-text ee-mod">
<span class="hide">
              industry
            </span>
<span>
<span class="info-company-name">
                Ventas y Servicios S.A.
              </span>
</span>
</h3>
</li>
<li>
<spa

In [64]:
def san(s):
    s = [re.sub("\\n|\\t|\\r","",x) for x in s]
    s = [re.sub("^ +","",x) for x in s]
    s = [re.sub(" +$","",x) for x in s]
    return(s)

titles = san([x.select(".text-ellipsis")[0].get_text() for x in jobs])
salaries = san([x.select(".info-salary")[0].get_text() for x in jobs])
cities = san([x.select(".info-city")[0].get_text() for x in jobs])
companies = san([x.select(".info-company-name")[0].get_text() for x in jobs])
dates = san([x.select(".info-publish-date")[0].get_text() for x in jobs])
links = [x.select("div a")[0]['href'] for x in jobs]

dates = [re.sub("^Publicado ","",x) for x in dates]

jobs_tab = pd.DataFrame({
        "date": dates, 
        "firm": companies, 
        "city": cities,
        "title": titles,
        "salary": salaries,
        "link": links
    })
jobs_tab

,city,date,firm,link,salary,title
0,Bogotá,3 Jun 2018,Ventas y Servicios S.A.,/co/ofertas-trabajo/ejecutivo-comercial-extern...,"$2,5 a $3 millones",Ejecutivo comercial externo grupo aval $951.000!!
1,Bogotá,3 Jun 2018,STAFFING DE COLOMBIA,/co/ofertas-trabajo/auxilaires-de-bodega/18834...,Menos de $1 millón,Auxilaires de bodega
2,Toda Colombia,3 Jun 2018,STAFFING DE COLOMBIA,/co/ofertas-trabajo/asesor-comercial-claro-en-...,"$1,5 a $2 millones",Asesor comercial claro en ubate
3,Bogotá,3 Jun 2018,STAFFING DE COLOMBIA,/co/ofertas-trabajo/mantenimiento-locativo-tod...,Menos de $1 millón,Mantenimiento locativo todero para contratació...
4,Bogotá,3 Jun 2018,TELEPERFORMANCE COLOMBIA S.A.S,/co/ofertas-trabajo/practicas-de-comunicacion-...,Menos de $1 millón,Prácticas de comunicación social
5,Bogotá,3 Jun 2018,SOLUCIONES INMEDIATAS S.A,/co/ofertas-trabajo/soldador-tig-o-mig/1883405462,"$1 a $1,5 millones",Soldador tig o mig
6,Bogotá,3 Jun 2018,Listos S.A.,/co/ofertas-trabajo/analista-de-seleccion/1883...,"$1 a $1,5 millones",Analista de selección
7,Bogotá,3 Jun 2018,STAFFING DE COLOMBIA,/co/ofertas-trabajo/gran-convocatoria-asesor-c...,"$1,5 a $2 millones",Gran convocatoria asesor call center migraciones
8,Bogotá,3 Jun 2018,SOLUCIONES INMEDIATAS S.A,/co/ofertas-trabajo/fresador-convencional-cnc/...,"$1 a $1,5 millones",Fresador convencional cnc
9,Bogotá,3 Jun 2018,TELEPERFORMANCE COLOMBIA S.A.S,/co/ofertas-trabajo/bachiller-meses-de-experie...,Menos de $1 millón,Bachiller 6 meses de experiencia en call center
